In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
37238,2021-01-28 16:58:59+00:00,2021-01-28 17:01:05+00:00,Production,0004 of 0004,P4,None,None,KL1,Friday Assembly,Thursday Afternoon,iXUsr_Selina(6751),4,2.10,2021-05
37241,2021-01-28 16:32:00+00:00,2021-01-28 16:34:41+00:00,Production,0002 of 0003,U4,None,None,KL1,Friday Assembly,Thursday Afternoon,iXUsr_Selina(6751),4,2.68,2021-05
9555,2021-01-30 07:28:24+00:00,2021-01-30 07:29:21+00:00,Production,None,B4,None,None,KL3,None,Saturday Morning,iXUsr_Gael(7447),5,0.95,2021-05
16975,2021-01-29 17:45:12+00:00,2021-01-29 17:48:59+00:00,Production,0033 of 0051,B2,None,None,KL3,Saturday Assembly,Friday Afternoon,iXUsr_Peni(8294),3,3.78,2021-05
13906,2021-01-28 09:02:10+00:00,2021-01-28 09:02:20+00:00,Production,0005 of 0017,D2,None,None,KL10,Friday Assembly,Thursday Morning,iXUsr_Gwen(7292),3,0.17,2021-05


In [5]:
data_in_use.shape

(63370, 14)